In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as	plt
import seaborn as sns
import json
import math

c:\Users\Ephyrus\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Part 1

$p\left(x_1, \ldots, x_n, y_1, \ldots, y_n\right)=\prod_{i=1}^{n+1} q\left(y_i \mid y_{i-1}\right) \cdot \prod_{i=1}^n e\left(x_i \mid y_i\right)$

In [2]:
df = pd.read_csv('train.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])

C:\Users\Ephyrus\AppData\Local\Temp\ipykernel_11972\2090610779.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('train.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])


In [3]:
display(df)

Token Label
0      Estuvimos     O
1           hace     O
2           poco     O
3             mi     O
4         pareja     O
...          ...   ...
31171      junto     O
31172        con     O
31173         la     O
31174      carta     O
31175          .     O

[31176 rows x 2 columns]

In [4]:
df_token = df.loc[:,["Token"]]
df_label = df.loc[:,["Label"]]

arr_token = (df_token.squeeze()).to_numpy()
arr_label = (df_label.squeeze()).to_numpy()

token_all = df_token.squeeze().unique()
label_all = df_label.squeeze().unique()

In [5]:
print(label_all)

['O' 'B-positive' 'B-negative' 'B-neutral' 'I-neutral' 'I-positive'
 'I-negative']


In [6]:
# y_all = pd.DataFrame(df_label.value_counts()).reset_index()
# y_all.columns = ['Label', 'Count']
# y_all["Location"] = [np.where(arr_label == i)[0] for i in label_all]

y_all = pd.DataFrame(label_all, columns=['Label'])
y_all["Location"] = [np.where(arr_label == i)[0] for i in label_all]
y_all["Count"] = [len(y_all.loc[i, "Location"]) for i in range(len(y_all))]

print(len(y_all))
print(type(y_all))
display(y_all)

7
<class 'pandas.core.frame.DataFrame'>


Label                                           Location  Count
0           O  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...  29035
1  B-positive  [16, 19, 22, 25, 60, 63, 75, 86, 90, 113, 204,...   1160
2  B-negative  [39, 98, 124, 131, 133, 137, 323, 382, 472, 62...    381
3   B-neutral  [153, 591, 685, 1053, 1180, 1467, 1973, 1994, ...     72
4   I-neutral  [154, 155, 156, 157, 158, 159, 160, 161, 162, ...     43
5  I-positive  [313, 314, 315, 937, 1400, 1401, 1939, 2054, 2...    314
6  I-negative  [802, 1171, 1626, 1627, 1724, 1725, 1726, 1727...    171

In [7]:
x_all = pd.DataFrame(token_all, columns=['Token'])

# Need to check more, this takes >1m to do
new_x_all = pd.DataFrame({'Token': np.repeat(x_all['Token'], 7)}).reset_index()
new_x_all.columns = ['Index', 'Token']
new_x_all["Label"] = np.tile(label_all, int(len(new_x_all)/7))
count_x_all = []
location_x_all = []

for i in token_all:
    for j in label_all:
        cond_1 = arr_token == i
        cond_2 = arr_label == j
        loc = (np.where(cond_1 & cond_2)[0]).tolist()
        # df_ij = df.iloc[loc, :]
        count_x_all += [len(loc)]
        location_x_all += [loc]
new_x_all["Count"] = count_x_all
new_x_all["Location"] = location_x_all

print(len(new_x_all))
print(type(new_x_all))
display(new_x_all)

35280
<class 'pandas.core.frame.DataFrame'>


Index        Token       Label  Count  \
0          0    Estuvimos           O      6   
1          0    Estuvimos  B-positive      0   
2          0    Estuvimos  B-negative      0   
3          0    Estuvimos   B-neutral      0   
4          0    Estuvimos   I-neutral      0   
...      ...          ...         ...    ...   
35275   5039  inolvidable  B-negative      0   
35276   5039  inolvidable   B-neutral      0   
35277   5039  inolvidable   I-neutral      0   
35278   5039  inolvidable  I-positive      0   
35279   5039  inolvidable  I-negative      0   

                                     Location  
0      [0, 13026, 13936, 17095, 27870, 29879]  
1                                          []  
2                                          []  
3                                          []  
4                                          []  
...                                       ...  
35275                                      []  
35276                                      []  
35277                                      []  
35278                                      []  
35279                                      []  

[35280 rows x 5 columns]

$1. \:\:\: e(x \mid y)=\frac{\operatorname{Count}(y \rightarrow x)}{\operatorname{Count}(y)}$

In [8]:
def emm_par(x, y):
    # x is an instance of Token
    # y is an instance of Label
    cond_1 = new_x_all["Token"] == x
    cond_2 = new_x_all["Label"] == y
    num = ((new_x_all[cond_1 & cond_2])["Count"]).values[0]
    cond_3 = y_all["Label"] == y
    denom = ((y_all[cond_3])["Count"]).values[0]
    return num/denom

In [9]:
# a = emm_par("Estuvimos", "O")
# print(a)

# b = emm_par("estuvimos", "O")
# print(b)

$2. \:\:\:e(x \mid y)=\left\{\begin{array}{cc}
\frac{\operatorname{count}(y \rightarrow x)}{\operatorname{Count}(y)+k} & \text { If the word token } x \text { appears in the training set } \\
\frac{k}{\operatorname{Count}(y)+k} & \text { If word token } x \text { is the special token \#UNK\# }
\end{array}\right.$

In [10]:
def emm_par_test(x, y):
    # x is an instance of Token
    # y is an instance of Label
    k = 1
    if x in new_x_all.loc[:, "Token"].unique():
        cond_1 = new_x_all["Token"] == x
        cond_2 = new_x_all["Label"] == y
        num = ((new_x_all[cond_1 & cond_2])["Count"]).values[0]
    else:
        num = k
    cond_3 = y_all["Label"] == y
    denom = ((y_all[cond_3])["Count"]).values[0] + k
    return num/denom

In [11]:
# a = emm_par_test(",", "I-negative")
# print(a)

# b = emm_par_test("Kontollllllllllll", "I-neutral")
# print(b)

In [12]:
emm_par_score_all = []
new_token_all = np.append(token_all, "#UNK#")
print(new_token_all)

['Estuvimos' 'hace' 'poco' ... 'redondeó' 'inolvidable' '#UNK#']


In [13]:
# Need to check more, this takes >2m to do
for i in token_all:
    emm_par_score = 0
    for j in label_all:
        emm_par_score = emm_par_test(i,j)
        emm_par_score_all += [emm_par_score]
print(emm_par_score_all)

[0.00020664003306240529, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0008954401432704229, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0018942003030720485, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0024796803967488635, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00044772007163521146, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0352665656426505, 0.0, 0.0, 0.0, 0.0, 0.015873015873015872, 0.011627906976744186, 0.0012398401983744318, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005813953488372093, 0.00034440005510400884, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00013776002204160352, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0039606006336961016, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005813953488372093, 0.013638242182118749, 0.0, 0.0, 0.0, 0.0, 0.006349206349206349, 0.0, 0.005682600909216145, 0.0008613264427217916, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05730816916930707, 0.0, 0.0, 0.0, 0.0, 0.006349206349206349, 0.01744186046511628, 0.0013431602149056344, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.026002204160352666, 0.0, 0.0, 0.0, 0.022727272727272728, 0.0380952380952381, 0.01744186046511628, 0.0011020801763328282, 0

In [14]:
new_x_all["e(x|y)"] = emm_par_score_all
display(new_x_all)

Index        Token       Label  Count  \
0          0    Estuvimos           O      6   
1          0    Estuvimos  B-positive      0   
2          0    Estuvimos  B-negative      0   
3          0    Estuvimos   B-neutral      0   
4          0    Estuvimos   I-neutral      0   
...      ...          ...         ...    ...   
35275   5039  inolvidable  B-negative      0   
35276   5039  inolvidable   B-neutral      0   
35277   5039  inolvidable   I-neutral      0   
35278   5039  inolvidable  I-positive      0   
35279   5039  inolvidable  I-negative      0   

                                     Location    e(x|y)  
0      [0, 13026, 13936, 17095, 27870, 29879]  0.000207  
1                                          []  0.000000  
2                                          []  0.000000  
3                                          []  0.000000  
4                                          []  0.000000  
...                                       ...       ...  
35275                                      []  0.000000  
35276                                      []  0.000000  
35277                                      []  0.000000  
35278                                      []  0.000000  
35279                                      []  0.000000  

[35280 rows x 6 columns]

$3. \:\:\:y^*=\underset{y}{\arg \max } \:e(x \mid y)$

In [15]:
df_y = pd.DataFrame()

emm_par_score_all = []
y_star_all = []
new_token_all = np.append(token_all, "#UNK#")
print(new_token_all)
print(len(new_token_all))

['Estuvimos' 'hace' 'poco' ... 'redondeó' 'inolvidable' '#UNK#']
5041


In [16]:
# Need to check more, this takes >2m to do
for i in new_token_all:
    emm_par_score = 0
    y_star = ""
    for j in label_all:
        emm_par_score_temp = emm_par_test(i,j)
        if emm_par_score_temp > emm_par_score:
            emm_par_score = emm_par_score_temp
            y_star = j
        else:
            pass
    y_star_all += [y_star]
    emm_par_score_all += [emm_par_score]

In [17]:
print(y_star_all)
print(len(y_star_all))
print(emm_par_score_all)
print(len(emm_par_score_all))

['O', 'O', 'O', 'O', 'O', 'O', 'I-negative', 'O', 'O', 'I-negative', 'O', 'O', 'O', 'O', 'I-positive', 'B-positive', 'I-neutral', 'B-negative', 'B-positive', 'B-positive', 'B-negative', 'O', 'O', 'I-negative', 'O', 'O', 'O', 'O', 'I-neutral', 'O', 'I-negative', 'O', 'O', 'O', 'I-negative', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-negative', 'O', 'O', 'B-neutral', 'O', 'B-neutral', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-neutral', 'O', 'O', 'O', 'O', 'O', 'O', 'B-negative', 'O', 'O', 'B-positive', 'O', 'O', 'O', 'O', 'O', 'O', 'B-negative', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-positive', 'O', 'I-positive', 'B-negative', 'I-neutral', 'O', 'O', 'B-negative', 'B-negative', 'O', 'O', 'B-negative', 'O', 'O', 'O', 'I-neutral', 'O', 'I-positive', 'O', 'B-neutral', 'I-neutral', 'I-neutral', 'I-neutral', 'I-neutral', 'I-neutral', 'I-neutral', 'O', 'O', 'I-negative', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-neutral', 'O', 'I-negative', 'I-negative', 'O', 'O',

In [18]:
df_y["Token"] = new_token_all
df_y["y*"] = y_star_all
df_y["Max e(x|y)"] = emm_par_score_all

display(df_y)

Token         y*  Max e(x|y)
0       Estuvimos          O    0.000207
1            hace          O    0.000895
2            poco          O    0.001894
3              mi          O    0.002480
4          pareja          O    0.000448
...           ...        ...         ...
5036    informado          O    0.000034
5037     almuerzo          O    0.000034
5038     redondeó          O    0.000034
5039  inolvidable          O    0.000034
5040        #UNK#  I-neutral    0.022727

[5041 rows x 3 columns]

In [19]:
df_y_dict = {row['Token']: (row['y*'], row['Max e(x|y)']) for row in df_y.to_dict(orient='records')}
print(df_y_dict)

{'Estuvimos': ('O', 0.00020664003306240529), 'hace': ('O', 0.0008954401432704229), 'poco': ('O', 0.0018942003030720485), 'mi': ('O', 0.0024796803967488635), 'pareja': ('O', 0.00044772007163521146), 'y': ('O', 0.0352665656426505), 'yo': ('I-negative', 0.005813953488372093), 'comiendo': ('O', 0.00034440005510400884), 'resultó': ('O', 0.00013776002204160352), 'todo': ('I-negative', 0.005813953488372093), 'muy': ('O', 0.013638242182118749), 'bien': ('O', 0.005682600909216145), ',': ('O', 0.05730816916930707), 'tanto': ('O', 0.0013431602149056344), 'la': ('I-positive', 0.0380952380952381), 'comida': ('B-positive', 0.14556416881998277), 'el': ('I-neutral', 0.022727272727272728), 'vino': ('B-negative', 0.015706806282722512), 'trato': ('B-positive', 0.03789836347975883), 'decoración': ('B-positive', 0.006029285099052541), '…': ('B-negative', 0.002617801047120419), 'nos': ('O', 0.005028240804518529), 'gustó': ('O', 0.0003788400606144097), 'mucho': ('I-negative', 0.005813953488372093), '.': ('O'

### Emission Probability Testing

In [20]:
# df_test = pd.read_csv('dev_out.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])

# Read the data line by line
with open('dev_out.txt', 'r') as f:
    lines = f.read().splitlines()

# Split each line into two parts and replace empty lines with [np.nan, np.nan]
data = [line.split(' ') if line else ["", ""] for line in lines]

# Convert the list of lists into a DataFrame
df_test_original = pd.DataFrame(data, columns=['Token', 'Label'])

In [21]:
df_test = df_test_original.copy()

df_test_token = df_test.loc[:,["Token"]]
df_test_label = df_test.loc[:,["Label"]]

arr_test_token = (df_test_token.squeeze()).to_numpy()
arr_test_label = (df_test_label.squeeze()).to_numpy()

token_test_all = df_test_token.squeeze().unique()
label_test_all = df_test_label.squeeze().unique()

In [22]:
test_pred = []
new_token = []
for i in arr_test_token:
    try:
        pred = df_y_dict[i][0]
        test_pred += [pred]
    except:
        if not i:
            pred = ""
        else:
            pred = df_y_dict["#UNK#"][0]    
        test_pred += [pred]
        new_token += [i]

In [23]:
df_test["Predicted Label (Part 1)"] = test_pred
display(df_test)

arr_test_pred_label = ((df_test.loc[:,["Predicted Label (Part 1)"]]).squeeze()).to_numpy()

Token       Label Predicted Label (Part 1)
0           Plato  B-positive               B-negative
1     degustación  I-positive               I-positive
2               :           O                        O
3              un           O                        O
4            poco           O                        O
...           ...         ...                      ...
4573      dejarte           O                I-neutral
4574           la           O               I-positive
4575      cartera           O                I-neutral
4576            .           O                        O
4577                                                  

[4578 rows x 3 columns]

In [24]:
df_out = df_test.loc[:, ["Token", "Predicted Label (Part 1)"]]
df_out.to_csv('EvalScript/dev.prediction1', sep=' ', index=False, header=False)

# Part 2

In [25]:
# df_test = pd.read_csv('dev_out.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])

# Read the data line by line
with open('train.txt', 'r') as f:
    lines = f.read().splitlines()

# Split each line into two parts and replace empty lines with [np.nan, np.nan]
data = [line.split(' ') if line else ["", ""] for line in lines]

# Convert the list of lists into a DataFrame
df_ss = pd.DataFrame(data, columns=['Token', 'Label (yi-1)'])

In [26]:
display(df_ss)

Token Label (yi-1)
0      Estuvimos            O
1           hace            O
2           poco            O
3             mi            O
4         pareja            O
...          ...          ...
33028        con            O
33029         la            O
33030      carta            O
33031          .            O
33032                        

[33033 rows x 2 columns]

In [27]:
df_label_ss = df_ss.loc[:,["Label (yi-1)"]]
arr_label_ss = (df_label_ss.squeeze()).to_numpy()

In [28]:
label_all_ss = np.insert(label_all, 0, 'START')
label_all_ss = np.append(label_all_ss, 'STOP')

In [29]:
y_all_ss = pd.DataFrame(label_all_ss, columns=['Label (yi-1)'])

In [30]:
display(df_label_ss)
display(arr_label_ss)
display(label_all_ss)
display(y_all_ss)

Label (yi-1)
0                O
1                O
2                O
3                O
4                O
...            ...
33028            O
33029            O
33030            O
33031            O
33032             

[33033 rows x 1 columns]

array(['O', 'O', 'O', ..., 'O', 'O', ''], dtype=object)

array(['START', 'O', 'B-positive', 'B-negative', 'B-neutral', 'I-neutral',
       'I-positive', 'I-negative', 'STOP'], dtype=object)

Label (yi-1)
0        START
1            O
2   B-positive
3   B-negative
4    B-neutral
5    I-neutral
6   I-positive
7   I-negative
8         STOP

In [31]:
new_x_all_ss = y_all_ss.copy()
new_x_all_ss.columns = ["Label"]
display(new_x_all_ss)

Label
0       START
1           O
2  B-positive
3  B-negative
4   B-neutral
5   I-neutral
6  I-positive
7  I-negative
8        STOP

In [32]:
new_y_all_ss = pd.DataFrame({'Label (yi-1)': np.repeat(y_all_ss['Label (yi-1)'], 9)}).reset_index()
new_y_all_ss.columns = ['Index', 'Label (yi-1)']
new_y_all_ss["Destination Label (yi)"] = np.tile(label_all_ss, int(len(new_y_all_ss)/9))
new_y_all_ss["Count (yi-1,yi)"] = np.zeros(len(new_y_all_ss), dtype=int)

previous_y = "START"
for i in arr_label_ss:
    if i == "":
        i = "STOP"
    current_y = i
    new_y_all_ss.loc[(new_y_all_ss['Label (yi-1)'] == previous_y) & (new_y_all_ss['Destination Label (yi)'] == current_y), 'Count (yi-1,yi)'] += 1
    if current_y == "STOP":
        current_y = "START"
    previous_y = current_y

display(new_y_all_ss)

Index Label (yi-1) Destination Label (yi)  Count (yi-1,yi)
0       0        START                  START                0
1       0        START                      O             1725
2       0        START             B-positive               97
3       0        START             B-negative               26
4       0        START              B-neutral                9
..    ...          ...                    ...              ...
76      8         STOP              B-neutral                0
77      8         STOP              I-neutral                0
78      8         STOP             I-positive                0
79      8         STOP             I-negative                0
80      8         STOP                   STOP                0

[81 rows x 4 columns]

In [33]:
y_all_ss_sums = [int(new_y_all_ss.iloc[i:i+9]['Count (yi-1,yi)'].sum()) for i in range(0, len(new_y_all_ss), 9)]
print(y_all_ss_sums)

new_x_all_ss["Count"] = y_all_ss_sums
new_x_all_ss.loc[new_x_all_ss["Label"] == "STOP", "Count"] = new_x_all_ss.loc[new_x_all_ss["Label"]=="START", "Count"].values[0]
display(new_x_all_ss)

[1857, 29035, 1160, 381, 72, 43, 314, 171, 0]


Label  Count
0       START   1857
1           O  29035
2  B-positive   1160
3  B-negative    381
4   B-neutral     72
5   I-neutral     43
6  I-positive    314
7  I-negative    171
8        STOP   1857

$2. \:\:\:q\left(y_i \mid y_{i-1}\right)=\frac{\operatorname{Count}\left(y_{i-1}, y_i\right)}{\operatorname{Count}\left(y_{i-1}\right)}$

In [34]:
def q_test(x, y):
    cond_1 = new_y_all_ss["Label (yi-1)"] == y
    cond_2 = new_y_all_ss["Destination Label (yi)"] == x
    num = ((new_y_all_ss[cond_1 & cond_2])["Count (yi-1,yi)"]).values[0]
    cond_3 = new_x_all_ss["Label"] == y
    denom = ((new_x_all_ss[cond_3])["Count"]).values[0]
    if not denom:
        return 0
    return num/denom

In [35]:
q_value_all = []
for i in label_all_ss:
    for j in label_all_ss:
        # print(i,j)
        q_value = q_test(j,i)
        q_value_all += [q_value]
print(q_value_all)

[0.0, 0.9289176090468497, 0.052234787291330104, 0.014001077005923533, 0.004846526655896607, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8856896848630963, 0.03650766316514551, 0.012226623041157224, 0.0021353538832443605, 0.0, 0.0, 0.0, 0.06344067504735663, 0.0, 0.871551724137931, 0.002586206896551724, 0.0, 0.0008620689655172414, 0.0, 0.11637931034482758, 0.0, 0.008620689655172414, 0.0, 0.8110236220472441, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1784776902887139, 0.010498687664041995, 0.0, 0.7916666666666666, 0.0, 0.0, 0.0, 0.20833333333333334, 0.0, 0.0, 0.0, 0.0, 0.3488372093023256, 0.0, 0.0, 0.0, 0.6511627906976745, 0.0, 0.0, 0.0, 0.0, 0.4267515923566879, 0.0, 0.0, 0.0, 0.0, 0.5700636942675159, 0.0, 0.0031847133757961785, 0.0, 0.39766081871345027, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6023391812865497, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [36]:
new_y_all_ss["q(yi|yi-1)"] = q_value_all


In [37]:
display(new_y_all_ss)

Index Label (yi-1) Destination Label (yi)  Count (yi-1,yi)  q(yi|yi-1)
0       0        START                  START                0    0.000000
1       0        START                      O             1725    0.928918
2       0        START             B-positive               97    0.052235
3       0        START             B-negative               26    0.014001
4       0        START              B-neutral                9    0.004847
..    ...          ...                    ...              ...         ...
76      8         STOP              B-neutral                0    0.000000
77      8         STOP              I-neutral                0    0.000000
78      8         STOP             I-positive                0    0.000000
79      8         STOP             I-negative                0    0.000000
80      8         STOP                   STOP                0    0.000000

[81 rows x 5 columns]

In [38]:
display(new_x_all)

Index        Token       Label  Count  \
0          0    Estuvimos           O      6   
1          0    Estuvimos  B-positive      0   
2          0    Estuvimos  B-negative      0   
3          0    Estuvimos   B-neutral      0   
4          0    Estuvimos   I-neutral      0   
...      ...          ...         ...    ...   
35275   5039  inolvidable  B-negative      0   
35276   5039  inolvidable   B-neutral      0   
35277   5039  inolvidable   I-neutral      0   
35278   5039  inolvidable  I-positive      0   
35279   5039  inolvidable  I-negative      0   

                                     Location    e(x|y)  
0      [0, 13026, 13936, 17095, 27870, 29879]  0.000207  
1                                          []  0.000000  
2                                          []  0.000000  
3                                          []  0.000000  
4                                          []  0.000000  
...                                       ...       ...  
35275                                      []  0.000000  
35276                                      []  0.000000  
35277                                      []  0.000000  
35278                                      []  0.000000  
35279                                      []  0.000000  

[35280 rows x 6 columns]

$3. \:\:\:y_1^*, \ldots, y_n^*=\underset{y_1, \ldots, y_n}{\arg \max } \:\:p\left(x_1, \ldots, x_n, y_1, \ldots, y_n\right)$

In [39]:
import numpy as np

def viterbi_algorithm(sent, transition_probs, emission_probs, tags, all_states):
    # Initialize the viterbi matrix and backpointer matrix
    viterbi = np.zeros((len(tags), len(sent)))
    backpointer = np.zeros((len(tags), len(sent)), 'int')

    # Initialization step
    for i, tag in enumerate(tags):
        if sent[0] not in all_states:
            viterbi[i, 0] = transition_probs[('START', 'I-neutral')]
        elif (tag, sent[0]) in emission_probs:
            viterbi[i, 0] = transition_probs[('START', tag)] * emission_probs[(tag, sent[0])]
        else:
            viterbi[i, 0] = transition_probs[('START', tag)] * 1e-10  # small number for unknown words

    # Recursion step
    for t in range(1, len(sent)):
        for i, tag in enumerate(tags):
            if sent[t] not in all_states:
                max_prob, best_tag = (viterbi[i, t-1] * transition_probs[(tags[i], 'I-neutral')], i)
            else:
                max_prob, best_tag = max((viterbi[j, t-1] * transition_probs[(tags[j], tag)] * emission_probs.get((tag, sent[t]), 1e-10), j) for j in range(len(tags)))
            viterbi[i, t] = max_prob
            backpointer[i, t] = best_tag

    # Termination step
    best_path_pointer = np.argmax([viterbi[i, -1] * transition_probs[(tags[i], 'STOP')] for i in range(len(tags))])
    best_path_prob = viterbi[best_path_pointer, -1] * transition_probs[(tags[best_path_pointer], 'STOP')]

    # Backtrack to get the best path
    best_path = []
    for t in range(len(sent)-1, -1, -1):
        best_path.insert(0, tags[best_path_pointer])
        best_path_pointer = backpointer[best_path_pointer, t]

    return best_path, best_path_prob

### Viterbi Algorithm Testing

In [40]:
def create_start_prob(df1, df2):
    count_values = df1.loc[df1['Label (yi-1)'] == 'START', 'Count (yi-1,yi)'].tolist()
    count_start = df2.loc[df2['Label'] == 'START', 'Count'].values[0]
    list1 = [value / count_start for value in count_values]
    list2 = label_all_ss.tolist()
    return dict(zip(list2, list1))

def create_trans_prob(df):
    trans_prob = {}
    for index, row in df.iterrows():
        yi_1 = row['Label (yi-1)']
        yi = row['Destination Label (yi)']
        prob = row['q(yi|yi-1)']
        if yi_1 not in trans_prob:
            trans_prob[yi_1] = {}
        trans_prob[yi_1][yi] = prob
    return trans_prob

def create_emit_prob(df):
    emit_prob = {}
    for index, row in df.iterrows():
        label = row['Label']
        token = row['Token']
        prob = row['e(x|y)']
        if label not in emit_prob:
            emit_prob[label] = {}
        emit_prob[label][token] = prob
    # Add "START" and "STOP" with all zero probabilities
    # emit_prob["START"] = {token: 0 for token in df['Token'].unique()}
    # emit_prob["STOP"] = {token: 0 for token in df['Token'].unique()}
    return emit_prob

def create_observed_states(df):
    big_list = []
    small_list = []
    for token in df['Token']:
        if token.strip() != '':
            small_list.append(token)
        else:
            big_list.append(small_list)
            small_list = []
    return big_list

def transform_dict(old_dict):
    new_dict = {}
    for key_outer in old_dict.keys():
        for key_inner in old_dict[key_outer].keys():
            new_key = (key_outer, key_inner)
            new_dict[new_key] = old_dict[key_outer][key_inner]
    return new_dict

def create_emit_probs(df):
    emit_prob = {}
    for index, row in df.iterrows():
        label = row['Label']
        token = row['Token']
        prob = row['e(x|y)']
        key = (label, token)
        emit_prob[key] = prob
    return emit_prob

In [41]:
all_states = token_all.tolist()
print(all_states)

sent = create_observed_states(df_test)
print(sent)

tags = ['O', 'B-positive', 'B-negative', 'B-neutral', 'I-neutral', 'I-positive', 'I-negative']
print(tags)

start_prob = create_start_prob(new_y_all_ss, new_x_all_ss)
print(start_prob)

trans_prob = create_trans_prob(new_y_all_ss)
print(trans_prob)

trans_probs = transform_dict(trans_prob)
print(trans_probs)

emit_prob = create_emit_prob(new_x_all)
print(emit_prob)

emit_probs = create_emit_probs(new_x_all)
print(emit_probs)

['Estuvimos', 'hace', 'poco', 'mi', 'pareja', 'y', 'yo', 'comiendo', 'resultó', 'todo', 'muy', 'bien', ',', 'tanto', 'la', 'comida', 'el', 'vino', 'trato', 'decoración', '…', 'nos', 'gustó', 'mucho', '.', 'Por', 'poner', 'algún', 'pero', 'quizá', 'jamón', 'no', 'era', 'lo', '"', 'ibérico', 'que', 'cabía', 'esperar', 'Bien', 'sabe', 'autor', 'del', 'blog', ')', 'Comida', 'exquisita', 'Restaurante', 'diferente', 'creativo', 'agradable', 'Si', 'has', 'probado', 'sus', 'carnes', 'te', 'estas', 'perdiendo', 'algo', 'grande', '!', 'En', 'resumen', 'bien-muy', 'servicio', 'correcto', 'profesional', '02-12-', '2012', 'elegimos', 'este', 'restaurante', 'por', 'los', 'comentarios', 'ha', 'sido', 'una', 'Grandisima', 'Decepción', 'Salimos', 'encantadas', 'Ubicación', 'Comimos', 'atendidos', 'Los', 'servicios', 'en', 'estado', 'lamentable', 'manteles', 'copas', 'etc', '<->', 'Gambas', 'resecas', 'sabemos', 'llenaron', 'con', 'trozos', 'de', 'esa', 'sucedanio', 'forma', 'palo', '-', 'sabor', 'a', '

{'O': {'Estuvimos': 0.00020664003306240529, 'hace': 0.0008954401432704229, 'poco': 0.0018942003030720485, 'mi': 0.0024796803967488635, 'pareja': 0.00044772007163521146, 'y': 0.0352665656426505, 'yo': 0.0012398401983744318, 'comiendo': 0.00034440005510400884, 'resultó': 0.00013776002204160352, 'todo': 0.0039606006336961016, 'muy': 0.013638242182118749, 'bien': 0.005682600909216145, ',': 0.05730816916930707, 'tanto': 0.0013431602149056344, 'la': 0.026002204160352666, 'comida': 0.0011020801763328282, 'el': 0.022110483537677365, 'vino': 0.0008265601322496211, 'trato': 0.00044772007163521146, 'decoración': 6.888001102080176e-05, '…': 0.0015498002479680396, 'nos': 0.005028240804518529, 'gustó': 0.0003788400606144097, 'mucho': 0.0018253202920512468, '.': 0.055896128943380634, 'Por': 0.0010332001653120264, 'poner': 0.00017220002755200442, 'algún': 0.00020664003306240529, 'pero': 0.006578041052486569, 'quizá': 0.00013776002204160352, 'jamón': 3.444000551040088e-05, 'no': 0.012708362033337925, '

In [42]:
all_best_path = []
all_best_path_prob = []

for i in sent:
    best_path, best_path_prob = viterbi_algorithm(i, trans_probs, emit_probs, tags, all_states)
    best_path += [""]
    all_best_path += best_path
    all_best_path_prob += [best_path_prob]
    
print(all_best_path)
print(all_best_path_prob)

['B-negative', 'I-negative', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '', 'O', 'O', 'O', 'B-positive', 'O', 'O', 'O', 'O', '', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'O', '', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'O', '', 'O', 'O', 'O', 'O', '', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'O', '', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negative', 'I-negativ

In [43]:
df_test["Predicted Label (Part 2)"] = all_best_path
display(df_test)

Token       Label Predicted Label (Part 1)  \
0           Plato  B-positive               B-negative   
1     degustación  I-positive               I-positive   
2               :           O                        O   
3              un           O                        O   
4            poco           O                        O   
...           ...         ...                      ...   
4573      dejarte           O                I-neutral   
4574           la           O               I-positive   
4575      cartera           O                I-neutral   
4576            .           O                        O   
4577                                                     

     Predicted Label (Part 2)  
0                  B-negative  
1                  I-negative  
2                           O  
3                           O  
4                           O  
...                       ...  
4573               I-negative  
4574               I-negative  
4575               I-negative  
4576                        O  
4577                           

[4578 rows x 4 columns]

In [44]:
df_out = df_test.loc[:, ["Token", "Predicted Label (Part 2)"]]
df_out.to_csv('EvalScript/dev.prediction2', sep=' ', index=False, header=False)

## Part 3

In [45]:

import heapq

def k_best_viterbi(observations, transition_probability, emission_probability, start_probability, states, all_states, k=1):
    # """
    # Implements the K-best Viterbi algorithm to determine the K most probable sequences of hidden states.
    
    # Parameters:
    # - observations: A sequence of observations.
    # - states: A set of possible hidden states.
    # - start_probability: A dictionary specifying the starting probabilities for each state.
    # - transition_probability: A dictionary specifying the transition probabilities between states.
    # - emission_probability: A dictionary specifying the probabilities of observations given states.
    # - k: The number of sequences to return.
    # - all_states: A set of all words that have been trained on.

    # Returns:
    # - A list of the k most probable sequences with their probabilities.
    # """
    
    # Initialize Viterbi and backpointer tables
    # if k==1:
    #     return  viterbi_algorithm(observations, transition_probability, emission_probability, states, all_states)
    V = [{} for _ in range(len(observations))]
    path = [{} for _ in range(len(observations))]

    # Initialization step
    for state in states:
        obs_prob = emission_probability[state][observations[0]] if observations[0] in all_states else 0
        V[0][state] = [start_probability[state] * obs_prob]
        path[0][state] = [[state if obs_prob > 0 else 'I-neutral']]

    # Run Viterbi for t > 0
    for t in range(1, len(observations)):
        for current_state in states:
            # If observation is OOV, set to I-neutral
            if observations[t] not in all_states:
                V[t][current_state] = [0] * k
                path[t][current_state] = [['I-neutral'] * (t + 1)] * k
                continue

            # Get the k-best paths for the current state
            all_probs_paths = []
            for previous_state in states:
                for i in range(k):
                    if t-1 < len(V) and previous_state in V[t-1] and i < len(V[t-1][previous_state]):
                        prob = V[t-1][previous_state][i] * transition_probability[previous_state][current_state] * emission_probability[current_state][observations[t]]
                        current_path = path[t-1][previous_state][i] + [current_state]
                        all_probs_paths.append((prob, current_path))
            
            # Sort and select the k-best paths for the current state
            all_probs_paths.sort(reverse=True)
            V[t][current_state] = [item[0] for item in all_probs_paths[:k]]
            path[t][current_state] = [item[1] for item in all_probs_paths[:k]]

    # Use a priority queue to get the k-best paths based on their probabilities
    k_best_paths = []
    for state in states:
        for i in range(k):
            if state in V[-1] and i < len(V[-1][state]):
                heapq.heappush(k_best_paths, (-V[-1][state][i], path[-1][state][i]))

    # Get top-k sequences
    k_best_sequences = []
    for _ in range(k):
        if k_best_paths:
            prob, seq = heapq.heappop(k_best_paths)
            k_best_sequences.append((seq, -prob))

    return k_best_sequences

### Kth Best Viterbi Algorithm Testing (K=2)

In [46]:
k = 2
all_best_paths = {key: [''] for key in range(1,k+1)}

for i in sent:
    best_paths = k_best_viterbi(i, trans_prob, emit_prob, start_prob, tags, all_states, k)
    for j in range(k):
        best_path = best_paths[j][0]
        best_path += [""]
        all_best_paths[j+1] += best_path

In [47]:
df_test_2 = df_test_original.copy()

for i in range(k):
    col = "Predicted Label (Part 3) K=" + str(i+1)
    label = all_best_paths[i+1]
    label = label[1:]
    df_test_2[col] = label
    
display(df_test_2)

Token       Label Predicted Label (Part 3) K=1  \
0           Plato  B-positive                   B-negative   
1     degustación  I-positive                   I-negative   
2               :           O                            O   
3              un           O                            O   
4            poco           O                            O   
...           ...         ...                          ...   
4573      dejarte           O                    I-neutral   
4574           la           O                    I-neutral   
4575      cartera           O                    I-neutral   
4576            .           O                   B-negative   
4577                                                         

     Predicted Label (Part 3) K=2  
0                      B-negative  
1                               O  
2                               O  
3                               O  
4                               O  
...                           ...  
4573                    I-neutral  
4574                    I-neutral  
4575                    I-neutral  
4576                   B-negative  
4577                               

[4578 rows x 4 columns]

In [48]:
for i in range(k):
    col = "Predicted Label (Part 3) K=" + str(i+1)
    df_out = df_test_2.loc[:, ["Token", col]]
    loc = "EvalScript/dev.prediction3_k_" + str(i+1)
    df_out.to_csv(loc, sep=' ', index=False, header=False)

### Kth Best Viterbi Algorithm Testing (K=8)

In [49]:
k = 8
all_best_paths = {key: [''] for key in range(1,k+1)}

for i in sent:
    best_paths = k_best_viterbi(i, trans_prob, emit_prob, start_prob, tags, all_states, k)
    for j in range(k):
        try:
            best_path = best_paths[j][0]
        except:
            best_path = best_paths[j-1][0]
        all_best_paths[j+1] += best_path
    for l in range(k):
        all_best_paths[l+1] += [""]

In [50]:
df_test_8 = df_test_original.copy()

for i in range(k):
    col = "Predicted Label (Part 3) K=" + str(i+1)
    label = all_best_paths[i+1]
    label = label[1:]
    df_test_8[col] = label
    
display(df_test_8)

Token       Label Predicted Label (Part 3) K=1  \
0           Plato  B-positive                   B-negative   
1     degustación  I-positive                   I-negative   
2               :           O                            O   
3              un           O                            O   
4            poco           O                            O   
...           ...         ...                          ...   
4573      dejarte           O                    I-neutral   
4574           la           O                    I-neutral   
4575      cartera           O                    I-neutral   
4576            .           O                   B-negative   
4577                                                         

     Predicted Label (Part 3) K=2 Predicted Label (Part 3) K=3  \
0                      B-negative                            O   
1                               O                            O   
2                               O                            O   
3                               O                            O   
4                               O                            O   
...                           ...                          ...   
4573                    I-neutral                    I-neutral   
4574                    I-neutral                    I-neutral   
4575                    I-neutral                    I-neutral   
4576                   B-negative                   B-negative   
4577                                                             

     Predicted Label (Part 3) K=4 Predicted Label (Part 3) K=5  \
0                      B-negative                   B-negative   
1                      I-negative                   I-negative   
2                               O                            O   
3                               O                            O   
4                               O                            O   
...                           ...                          ...   
4573                    I-neutral                    I-neutral   
4574                    I-neutral                    I-neutral   
4575                    I-neutral                    I-neutral   
4576                   B-negative                   B-negative   
4577                                                             

     Predicted Label (Part 3) K=6 Predicted Label (Part 3) K=7  \
0                      B-negative                            O   
1                               O                            O   
2                               O                            O   
3                               O                            O   
4                               O                            O   
...                           ...                          ...   
4573                    I-neutral                    I-neutral   
4574                    I-neutral                    I-neutral   
4575                    I-neutral                    I-neutral   
4576                   B-negative                   B-negative   
4577                                                             

     Predicted Label (Part 3) K=8  
0                      B-negative  
1                               O  
2                               O  
3                               O  
4                               O  
...                           ...  
4573                    I-neutral  
4574                    I-neutral  
4575                    I-neutral  
4576                   B-negative  
4577                               

[4578 rows x 10 columns]

In [51]:
for i in range(k):
    col = "Predicted Label (Part 3) K=" + str(i+1)
    df_out = df_test_8.loc[:, ["Token", col]]
    loc = "EvalScript/dev.prediction3_k_" + str(i+1)
    df_out.to_csv(loc, sep=' ', index=False, header=False)